Copyright (c) 2020 ARS Computer & Consulting GmbH

This program and the accompanying materials are made
available under the terms of the Eclipse Public License 2.0
which is available at https://www.eclipse.org/legal/epl-2.0/

SPDX-License-Identifier: EPL-2.0


Please note: this notebook uses _ffmpeg_ as external tool, in a different process. Make sure you have the executable at the right place or adapt the path is the respective code cell. You may download ffmpeg from the projects web site at https://ffmpeg.org/ .



In [9]:
import subprocess
import io, os
import time, datetime

CAPTURE_INTERVAL = 1 * 60 # 1 min
PROJECT_INTERVAL = 2 * 60 * 60 # 2 hour

In [11]:
def captureAFrame(cameraName, projectName, photoID):
    captureFrame = subprocess.run(
        ["../video_capturer/ffmpeg/bin/ffmpeg.exe", "-n", "-hide_banner", "-f", "dshow", "-i", 
         "video=" + cameraName, "-vframes", "1", projectName + "/" + str(photoID) + ".png"],
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE)

    if captureFrame.returncode == 0:
        return True, None
    else:
        return False, io.BytesIO(captureFrame.stderr)

In [12]:
#1. get the list of cameras

cameraInfo = subprocess.run(
    ["../video_capturer/ffmpeg/bin/ffmpeg.exe", "-hide_banner", "-list_devices", "true", "-f", "dshow", "-i", "dummyffmpeg"], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE)

#print(cameraInfo.returncode)

outputStream = io.BytesIO(cameraInfo.stderr)
output = outputStream.readline()

while len(output) > 0:
    trimedOutput = output.decode().strip()
    if trimedOutput.startswith("[dshow @"):
        print(trimedOutput)
    
    output = outputStream.readline()

[dshow @ 000001b728a29840] DirectShow video devices (some may be both video and audio devices)
[dshow @ 000001b728a29840]  "Lenovo EasyCamera"
[dshow @ 000001b728a29840]     Alternative name "@device_pnp_\\?\usb#vid_05e3&pid_0510&mi_00#9&ab0188f&0&0000#{65e8773d-8f56-11d0-a3b9-00a0c9223196}\global"
[dshow @ 000001b728a29840] DirectShow audio devices
[dshow @ 000001b728a29840]  "Microphone (JBL Reflect Aware Audio Interface)"
[dshow @ 000001b728a29840]     Alternative name "@device_cm_{33D9A762-90C8-11D0-BD43-00A0C911CE86}\wave_{EA69D82F-2E1C-4D2E-A007-2415D599F72E}"


In [13]:
# 2. chooose a new project name
newProjectName = input("Choose a name for the new project: ")
while True: 
    newProjectName = newProjectName.strip()
    if len(newProjectName) == 0:
        newProjectName = input("Choose a name for the new project: ")
    elif os.path.isdir(newProjectName):
        newProjectName = input("Given project is already exists, choose a name for the new project: ")
    else:
        os.mkdir(newProjectName)
        print("Project", newProjectName, "initialized.")
        break
    

Choose a name for the new project: test2
Project test2 initialized.


In [14]:
# 3. setting parameters

captureInterval = input("How often should a photo be taken (unit: seconds/default: 60 seconds)?")
projectInterval = input("How long should the experiment be (unit:seconds/default: 7200 seconds)?")

try:
    CAPTURE_INTERVAL = int(captureInterval)
except:
    pass

try:
    PROJECT_INTERVAL = int(projectInterval)
except:
    pass

How often should a photo be taken (unit: seconds/default: 60 seconds)?
How long should the experiment be (unit:seconds/default: 7200 seconds)?


In [15]:
# 4.camera setup and start
# NOTE!! if you want to interrupt the recording process. You can press the "interrupt the kernel" button. 
# However, you may need to wait at most "captureInterval" before kernel being interrupted. Please just be patient.

cameraName = input("Which camera would you like to use (e.g. Lenovo EasyCamera)? ")

while True:
    cameraName = cameraName.strip()
    if len(cameraName) > 0:
        break
        
    else:
        cameraName = input("Which camera would you like to use (e.g. Lenovo EasyCamera)? ")
    
for index in range(PROJECT_INTERVAL // CAPTURE_INTERVAL):
    result, err = captureAFrame(cameraName, newProjectName, index)
    
    if result:
        print("A Photo was succesfully taken at", datetime.datetime.now())
    else:
        print("Photo taken was failed. Debug messages:")
        
        output = err.readline()

        while len(output) > 0:
            trimedOutput = output.decode().strip()
            print(trimedOutput)
            
            output = err.readline()
    
    time.sleep(CAPTURE_INTERVAL)
        

Which camera would you like to use (e.g. Lenovo EasyCamera)? Lenovo EasyCamera
A Photo was succesfully taken at 2020-02-20 11:24:21.466423
A Photo was succesfully taken at 2020-02-20 11:25:22.777877
A Photo was succesfully taken at 2020-02-20 11:26:24.089350
A Photo was succesfully taken at 2020-02-20 11:27:25.400972


KeyboardInterrupt: 